In [28]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
#import seaborn
#seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
import matplotlib.ticker
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 1000) 

## Reading in the data files

In [29]:
#read in the data
df= pd.read_csv("vaclog", sep='\t', index_col=False)
df.drop(columns=["I_emission","I_grid","T-platinum"], inplace=True)


In [30]:
#add elapsed time

df_timestamps=pd.to_datetime(df["Time"],format="%d/%m/%Y %H:%M:%S")

runtime=(df_timestamps-df_timestamps[0]).dt.total_seconds()    
df.insert(1,"Elapsed time",runtime)
df


,Live comments,Elapsed time,Time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
0,NaN,0.0,01/03/2023 12:11:24,78.324,5.810000e-10,4.300000e-10,4.996000e-09,5.337000e-07,1.1,0.0,88.776
1,NaN,7.0,01/03/2023 12:11:31,78.325,5.660000e-10,4.240000e-10,4.996000e-09,5.324000e-07,1.1,0.0,88.888
2,NaN,14.0,01/03/2023 12:11:38,78.324,5.650000e-10,4.250000e-10,4.996000e-09,5.305000e-07,1.1,0.0,88.664
3,NaN,20.0,01/03/2023 12:11:44,78.324,5.640000e-10,4.240000e-10,4.996000e-09,5.292000e-07,1.1,0.0,88.888
4,NaN,27.0,01/03/2023 12:11:51,78.324,5.630000e-10,4.230000e-10,4.996000e-09,5.267000e-07,1.1,0.0,88.776
...,...,...,...,...,...,...,...,...,...,...,...
2066,NaN,14108.0,01/03/2023 16:06:32,77.501,7.530000e-09,8.130000e-09,5.179000e-09,3.754000e-07,1.1,260.0,4297.177
2067,NaN,14115.0,01/03/2023 16:06:39,77.501,7.470000e-09,8.060000e-09,5.081000e-09,3.754000e-07,1.1,260.0,4297.065
2068,NaN,14122.0,01/03/2023 16:06:46,77.501,7.450000e-09,8.000000e-09,5.050000e-09,3.754000e-07,1.1,260.0,4297.177
2069,NaN,14129.0,01/03/2023 16:06:53,77.501,7.400000e-09,7.980000e-09,5.154000e-09,3.754000e-07,1.1,259.0,4297.289


In [31]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
#hv["I_em"]=abs(hv["I_em"])
#hv["hv_grid"]=abs(hv["hv_grid"])
hv["datetime"] = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
#hv['datetime'] = hv['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
#hv.insert(0,"datetime",hv["Date"]+" "+hv["Time"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps1=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps1-timestamps1[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)


hv



,Date,Time,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,hv_grid,I_em,datetime
0,01/03/2023,14:27:53.522,0.000,0.0,0.08,0.06,0.08,0.0,0.00,0.0,...,0.0064,0.0085,0.0098,0.0099,-0.0050,-0.0141,NaN,9.700000e-09,8.400000e-09,2023-03-01 14:27:53.522
1,01/03/2023,14:27:54.706,1.184,0.0,0.08,0.06,0.08,0.0,0.00,0.0,...,0.0064,0.0085,0.0098,0.0099,-0.0050,-0.0141,NaN,9.700000e-09,8.400000e-09,2023-03-01 14:27:54.706
2,01/03/2023,14:27:55.883,2.361,0.0,0.08,0.06,0.08,0.0,0.00,0.0,...,0.0064,0.0085,0.0098,0.0099,-0.0050,-0.0141,NaN,9.700000e-09,8.400000e-09,2023-03-01 14:27:55.883
3,01/03/2023,14:27:57.068,3.546,0.0,0.08,0.06,0.08,0.0,0.00,0.0,...,0.0064,0.0085,0.0098,0.0099,-0.0050,-0.0141,NaN,9.700000e-09,8.400000e-09,2023-03-01 14:27:57.068
4,01/03/2023,14:27:58.248,4.726,0.0,0.08,0.06,0.08,0.0,0.00,0.0,...,0.0064,0.0085,0.0098,0.0099,-0.0050,-0.0141,NaN,9.700000e-09,8.400000e-09,2023-03-01 14:27:58.248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5017,01/03/2023,16:07:46.878,5993.356,0.0,100.08,1000.12,1000.24,1000.2,500.02,0.0,...,18.2871,18.3117,18.2933,18.2838,-0.0037,-0.0143,NaN,-5.636000e-07,3.995500e-06,2023-03-01 16:07:46.878
5018,01/03/2023,16:07:48.081,5994.559,0.0,100.08,1000.12,1000.24,1000.2,500.02,0.0,...,18.2887,18.3136,18.2958,18.2870,-0.0037,-0.0143,NaN,-5.631000e-07,3.995500e-06,2023-03-01 16:07:48.081
5019,01/03/2023,16:07:49.282,5995.760,0.0,100.08,1000.12,1000.24,1000.2,500.02,0.0,...,18.2891,18.3142,18.2962,18.2866,-0.0037,-0.0143,NaN,-5.653000e-07,3.995500e-06,2023-03-01 16:07:49.282
5020,01/03/2023,16:07:50.477,5996.955,0.0,100.08,1000.12,1000.24,1000.2,500.02,0.0,...,18.2896,18.3142,18.2962,18.2856,-0.0037,-0.0143,NaN,-5.619000e-07,3.996800e-06,2023-03-01 16:07:50.477


In [32]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Plots for debugging and data manipulation

In [33]:
#plotting parameters
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 10,
        }

plt.rcParams["figure.figsize"] = [8, 5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35
plt.rcParams["axes.grid.axis"] = "both"
plt.rcParams["axes.grid.which"] = "both"



In [34]:
#find the timestamp in vaclog, where emission started
#emission on: 1/03/2023,15:04:19.430
x = pd.Timestamp(2023, 3, 1, 15,4,19)

#then find the time value closest to it in the vaclog scan file 
print("\n","Result 1:",df.iloc[(df_timestamps - x).abs().argsort(),:])
print("\n","Result 2:",hv.iloc[(hv["datetime"] - x).abs().argsort(),:])



 Result 1:      Live comments  Elapsed time                 Time  injection 100mbar  \
1519           NaN       10372.0  01/03/2023 15:04:16             78.145   
1520           NaN       10379.0  01/03/2023 15:04:23             78.144   
1518           NaN       10365.0  01/03/2023 15:04:09             78.144   
1521           NaN       10386.0  01/03/2023 15:04:30             78.144   
1522           NaN       10392.0  01/03/2023 15:04:36             78.144   
...            ...           ...                  ...                ...   
4              NaN          27.0  01/03/2023 12:11:51             78.324   
3              NaN          20.0  01/03/2023 12:11:44             78.324   
2              NaN          14.0  01/03/2023 12:11:38             78.324   
1              NaN           7.0  01/03/2023 12:11:31             78.325   
0              NaN           0.0  01/03/2023 12:11:24             78.324   

          Barion_2      Barion_1  DUAL experiment  DUAL insulation  \
1519 

----------------------------------

In [35]:
#slicing the hv dataframe 
#include only the part with electron activity

EM = hv[1850:5021]

#slicing the df dataset
#include only the part with electron activity

esd = df.iloc[list((df["Elapsed time"]).between(10372,14135))]
print(esd)

     Live comments  Elapsed time                 Time  injection 100mbar  \
1519           NaN       10372.0  01/03/2023 15:04:16             78.145   
1520           NaN       10379.0  01/03/2023 15:04:23             78.144   
1521           NaN       10386.0  01/03/2023 15:04:30             78.144   
1522           NaN       10392.0  01/03/2023 15:04:36             78.144   
1523           NaN       10399.0  01/03/2023 15:04:43             78.144   
1524           NaN       10406.0  01/03/2023 15:04:50             78.144   
1525           NaN       10413.0  01/03/2023 15:04:57             78.144   
1526           NaN       10420.0  01/03/2023 15:05:04             78.144   
1527           NaN       10426.0  01/03/2023 15:05:10             78.144   
1528           NaN       10433.0  01/03/2023 15:05:17             78.145   
1529           NaN       10440.0  01/03/2023 15:05:24             78.144   
1530           NaN       10447.0  01/03/2023 15:05:31             78.145   
1531        

In [36]:
#plotting the sliced data
#Plotting the general pressure evolution for overview

fig, ax = plt.subplots()

ax.plot((esd["Elapsed time"]-10372)/3600,esd['Barion_1']/0.46,color="cyan",marker=".", markersize=4,label='BA1')
ax.plot((esd["Elapsed time"]-10372)/3600,esd['Barion_2']/0.46,color="red",marker=".", markersize=4,label='BA2')
plt.legend(loc="upper right")
#plotting the sliced HiVolta log
ax2=ax.twinx()
ax2.plot((EM["Elapsed time"]-2186)/3600,EM["I_em"],marker=".", markersize=4,label='Emission current')
#ax2.plot(EM["Elapsed time"]/3600,EM["hv_grid"],marker=".", markersize=4,label='Grid current')

ax.set_xlabel('Elapsed time (s)')
ax.set_ylabel('Pressure (mbar)')
ax2.set_ylabel('Current (A)')
ax.set_yscale('linear')

plt.legend(loc="upper right")
plt.title("Pressure and current evolution during ESD. H2 equivalent")


Text(0.5, 1.0, 'Pressure and current evolution during ESD. H2 equivalent')

In [37]:
plt.figure(figsize=(12,6))
plt.plot(EM["Elapsed time"]/3600,EM["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(EM["Elapsed time"]/3600,EM["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution during electron emission")


plt.yscale('log')


In [38]:
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

l5 = ax.plot(df["Elapsed time"]/3600, polyfit(A,df["T-CERNOX"]),marker=".", color="cyan", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Elapsed time (h)')
ax2=ax.twinx()

l1 = ax2.plot((df["Elapsed time"]/3600),df['Barion_1']/0.46,marker=".", markersize=5,label='BA1')
l2 = ax2.plot((df["Elapsed time"]/3600),df['Barion_2']/0.46,marker=".", markersize=5,label='BA2')
l3 = ax2.plot((df["Elapsed time"]/3600),df['DUAL experiment'],marker=".", markersize=5,label='DUAL')
ax2.set_ylabel('Pressure (mbar)')

ax2.legend(handles = l1+l2+l3+l5, loc="upper left")
plt.title("Pressure evolution + temperatures. H2 equivalent.")
ax2.set_yscale('log')


In [39]:
#TODO - calculating coverages for plotting pressure v coverage
#in the first assumption coverage is assumed to be equivalent to the number of molecules injected 
#coverage at any given time point in [M/cm2] -> M/s * total injection time 
#First - Determine the inj start point, set elapsed time here to 0. 
inj = df.loc[df["Barion_2"] >= 9e-8]
inj.drop(columns="Elapsed time", inplace=True)
print(inj)
timestamps2 = pd.to_datetime(inj["Time"],format="%d/%m/%Y %H:%M:%S")
runtime2 = (timestamps2-timestamps2[1265]).dt.total_seconds()    
inj.insert(1,"Elapsed time",runtime2)



     Live comments                 Time  injection 100mbar      Barion_2  \
1265           NaN  01/03/2023 14:35:18             78.412  2.180000e-07   
1266           NaN  01/03/2023 14:35:25             78.408  2.960000e-07   
1267           NaN  01/03/2023 14:35:32             78.408  1.870000e-07   
1268           NaN  01/03/2023 14:35:38             78.408  1.500000e-07   
1269           NaN  01/03/2023 14:35:45             78.407  1.490000e-07   
1270           NaN  01/03/2023 14:35:52             78.404  1.490000e-07   
1271           NaN  01/03/2023 14:35:59             78.404  1.490000e-07   
1272           NaN  01/03/2023 14:36:06             78.404  1.490000e-07   
1273           NaN  01/03/2023 14:36:12             78.402  1.480000e-07   
1274           NaN  01/03/2023 14:36:19             78.399  1.480000e-07   
1275           NaN  01/03/2023 14:36:26             78.400  1.480000e-07   
1276           NaN  01/03/2023 14:36:33             78.399  1.480000e-07   
1277        

C:\Users\etiirine\AppData\Local\Temp\ipykernel_19284\1870170200.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj.drop(columns="Elapsed time", inplace=True)


## Subtracting baseline

In [40]:
#TODO - subtract the baseline from graph 2 (linear pressure scale) and plot again
#adding H2 equiv pressure readings
inj.insert(3, "corr BA2", inj["Barion_2"]/0.46)
inj.insert(4, "corr BA1", inj["Barion_1"]/0.46)


### Thermal transpiration
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [41]:
#implementing the formula
T2 = 4.2
T1 = 293
p_coef = np.sqrt(T2/T1)
inj.insert(3, "H2 4K BA2", (inj["corr BA2"]*p_coef))
inj.insert(4, "H2 4K BA1", (inj["corr BA1"]*p_coef))


In [42]:
print(p_coef)
inj.head()

0.11972665112564158


,Live comments,Elapsed time,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
1265,NaN,0.0,01/03/2023 14:35:18,5.674002e-08,5.778112e-08,4.739130e-07,4.826087e-07,78.412,2.180000e-07,2.220000e-07,2.040000e-07,3.548000e-07,1.1,291.0,4274.057
1266,NaN,7.0,01/03/2023 14:35:25,7.704150e-08,7.808260e-08,6.434783e-07,6.521739e-07,78.408,2.960000e-07,3.000000e-07,1.474000e-07,3.544000e-07,1.1,291.0,4274.169
1267,NaN,14.0,01/03/2023 14:35:32,4.867149e-08,4.919204e-08,4.065217e-07,4.108696e-07,78.408,1.870000e-07,1.890000e-07,1.399000e-07,3.544000e-07,1.1,291.0,4274.169
1268,NaN,20.0,01/03/2023 14:35:38,3.904130e-08,3.956185e-08,3.260870e-07,3.304348e-07,78.408,1.500000e-07,1.520000e-07,1.106000e-07,3.544000e-07,1.1,291.0,4274.169
1269,NaN,27.0,01/03/2023 14:35:45,3.878102e-08,3.930157e-08,3.239130e-07,3.282609e-07,78.407,1.490000e-07,1.510000e-07,1.103000e-07,3.544000e-07,1.1,291.0,4274.506


In [43]:
#define baseline
base1 = inj[inj["corr BA1"].between(1e-7,4e-7)]
base2 = inj[inj["corr BA2"].between(1e-7,4e-7)]

#take the average of the baseline and apply thermal transpiration coef to these values
avg2 = np.mean(base1["corr BA2"])*p_coef
avg1 = np.mean(base2["corr BA1"])*p_coef
print(avg1)
print(avg2)


3.8429045398471315e-08
3.794746861184183e-08


In [44]:
#subtract the baseline from the graph to get the actual pressure evolution on the sample

#Plotting the h2 equiv base pressure subtracted data
fig, ax = plt.subplots()
#subtract the baseline from the graph to get the actual pressure evolution on the sample

ax.plot((inj["Elapsed time"]/3600),inj['H2 4K BA1'].sub(avg1),marker=".", markersize=5,label='BA1')
ax.plot((inj["Elapsed time"]/3600),inj['H2 4K BA2'].sub(avg2),marker=".", markersize=5,label='BA2')
plt.legend(loc="upper left")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

plt.legend(loc="upper right")
plt.title("Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ")


Text(0.5, 1.0, 'Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ')

### Injection pressure linearity

In [45]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = inj["injection 100mbar"].values
x = (inj["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.50*x+78.38


### Formula for sum of the differential injection flux
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

In [46]:
#constants in SI units
T = 293
k_B = 1.38e-23
V_inj = 6.515e-5
S_sample= 276
#implement the formula above
dp = abs(np.diff(inj["injection 100mbar"]*100))
print(np.mean(dp))
#dt = np.diff(inj["Elapsed time"])
#dif = abs(dp)*V_inj/dt
dp = np.insert(dp,0,0)
N_x = (dp.cumsum())*V_inj/(k_B*T)
print(N_x)
N_cov = (N_x/S_sample)
print(N_cov)


0.2781976744185909
[0.00000000e+00 6.44507098e+15 6.44507098e+15 6.44507098e+15
 8.05633872e+15 1.28901420e+16 1.28901420e+16 1.28901420e+16
 1.61126774e+16 2.09464807e+16 2.25577484e+16 2.41690162e+16
 2.90028194e+16 2.90028194e+16 2.90028194e+16 3.54478904e+16
 3.70591581e+16 4.18929614e+16 4.18929614e+16 4.83380323e+16
 4.83380323e+16 5.31718356e+16 5.47831033e+16 5.63943711e+16
 6.12281743e+16 6.12281743e+16 6.12281743e+16 6.12281743e+16
 6.92845130e+16 6.92845130e+16 7.41183163e+16 7.41183163e+16
 7.57295840e+16 7.73408518e+16 8.53971905e+16 8.70084582e+16
 8.70084582e+16 8.70084582e+16 9.34535292e+16 9.34535292e+16
 9.34535292e+16 9.98986002e+16 9.98986002e+16 1.04732403e+17
 1.06343671e+17 1.07954939e+17 1.19233813e+17 1.19233813e+17
 1.27290152e+17 1.28901420e+17 1.35346491e+17 1.35346491e+17
 1.35346491e+17 1.43402829e+17 1.43402829e+17 1.45014097e+17
 1.51459168e+17 1.51459168e+17 1.51459168e+17 1.57904239e+17
 1.57904239e+17 1.57904239e+17 1.57904239e+17 1.64349310e+17
 1.64

In [47]:
inj.insert(2, "Coverage",N_cov)


In [48]:
inj.head()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
1265,NaN,0.0,0.000000e+00,01/03/2023 14:35:18,5.674002e-08,5.778112e-08,4.739130e-07,4.826087e-07,78.412,2.180000e-07,2.220000e-07,2.040000e-07,3.548000e-07,1.1,291.0,4274.057
1266,NaN,7.0,2.335171e+13,01/03/2023 14:35:25,7.704150e-08,7.808260e-08,6.434783e-07,6.521739e-07,78.408,2.960000e-07,3.000000e-07,1.474000e-07,3.544000e-07,1.1,291.0,4274.169
1267,NaN,14.0,2.335171e+13,01/03/2023 14:35:32,4.867149e-08,4.919204e-08,4.065217e-07,4.108696e-07,78.408,1.870000e-07,1.890000e-07,1.399000e-07,3.544000e-07,1.1,291.0,4274.169
1268,NaN,20.0,2.335171e+13,01/03/2023 14:35:38,3.904130e-08,3.956185e-08,3.260870e-07,3.304348e-07,78.408,1.500000e-07,1.520000e-07,1.106000e-07,3.544000e-07,1.1,291.0,4274.169
1269,NaN,27.0,2.918963e+13,01/03/2023 14:35:45,3.878102e-08,3.930157e-08,3.239130e-07,3.282609e-07,78.407,1.490000e-07,1.510000e-07,1.103000e-07,3.544000e-07,1.1,291.0,4274.506


## Plotting

In [49]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot((inj["Elapsed time"]/3600),inj["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((inj["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 100mbar conductance gauge')

In [50]:
#Plotting BA2, BA1 pressure
fig, ax = plt.subplots()

ax.plot(df["Time"],df["Barion_2"],marker=".", markersize=4,label='Barion 2')
ax.plot(df["Time"],df["Barion_1"],marker=".", markersize=4,label='Barion 1')
ax.plot(df["Time"],df["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')
ax.plot(df["Time"],df['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.legend()
ax.set_title("S4 CU ESD sample out - Total pressure evolution")

#ax.grid(which='major', axis='y')
#ax.get_yaxis().get_major_formatter().labelOnlyBase = False
#ax.tick_params(axis="y", which='major')
ax.set_yscale('log')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.savefig(r'./graphs/S5 a-C H2 adsorption isotherm_Pressures.png',dpi=300)
plt.show()

In [51]:
#plot coverage vs time
fig, ax = plt.subplots()

ax.plot(inj["Time"],inj["Coverage"],marker=".", markersize=5,label='Coverage')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('Coverage (M/s*cm2)')
ax.set_yscale('linear')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.title("Coverage over time")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S5 a-C H2 ads isotherm_coverage vs time.png',dpi=300)
plt.show()

In [52]:
## dropping coverage values close to 0 (from before injection)
inj_new = inj.loc[inj['Coverage'] > 1e+13]
inj_new.head()
#inj_new["H2 4K BA2"].sub(avg2).describe()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-CERNOX
1266,NaN,7.0,2.335171e+13,01/03/2023 14:35:25,7.704150e-08,7.808260e-08,6.434783e-07,6.521739e-07,78.408,2.960000e-07,3.000000e-07,1.474000e-07,3.544000e-07,1.1,291.0,4274.169
1267,NaN,14.0,2.335171e+13,01/03/2023 14:35:32,4.867149e-08,4.919204e-08,4.065217e-07,4.108696e-07,78.408,1.870000e-07,1.890000e-07,1.399000e-07,3.544000e-07,1.1,291.0,4274.169
1268,NaN,20.0,2.335171e+13,01/03/2023 14:35:38,3.904130e-08,3.956185e-08,3.260870e-07,3.304348e-07,78.408,1.500000e-07,1.520000e-07,1.106000e-07,3.544000e-07,1.1,291.0,4274.169
1269,NaN,27.0,2.918963e+13,01/03/2023 14:35:45,3.878102e-08,3.930157e-08,3.239130e-07,3.282609e-07,78.407,1.490000e-07,1.510000e-07,1.103000e-07,3.544000e-07,1.1,291.0,4274.506
1270,NaN,34.0,4.670341e+13,01/03/2023 14:35:52,3.878102e-08,3.930157e-08,3.239130e-07,3.282609e-07,78.404,1.490000e-07,1.510000e-07,1.100000e-07,3.544000e-07,1.1,291.0,4274.281


In [53]:
#plotting pressure vs coverage
fig, ax = plt.subplots()

ax.plot((inj_new["Coverage"]),inj_new["H2 4K BA2"].sub(avg2),marker=".", markersize=5,label='isotherm')
ax.legend(loc="upper left")
#ax.set_xlim(left = 1e16,right = 3.119e16)

ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')
ax.set_xlim(1e15,6e15)

plt.title("H2 adsorption isotherm")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S5 a-C H2 adsorption isotherm.png',dpi=300)
plt.show()





## Writing edited dataframes to file


In [54]:
#desorption yield params
#vaclog
df.to_csv(r'./calculated datasets/vaclogger_output.csv', sep='\t', encoding='utf-8', header=True)

#CERNoX
temps_dict = {"Timestamp":df["Time"],"Temperatures":polyfit(A,df["T-CERNOX"])}
temps = pd.DataFrame(temps_dict)
temps.to_csv(r'./calculated datasets/CERNoX temps.csv', sep='\t', encoding='utf-8', header=True)
#coverage
inj.to_csv(r'./calculated datasets/vaclog during injection.csv', sep='\t', encoding='utf-8', header=True)

#coverage > 1e13
inj_new.to_csv(r'./calculated datasets/cut vaclog during injection.csv', sep='\t', encoding='utf-8', header=True)


NameError: name 'vaclog' is not defined